In [89]:
import pandas as pd
import numpy as np
import limix
from load_genotypes import load_genotypes_plink
from generate_kinship import generate_kinship

geno_prefix = '../data/geuvadis_test_data/Geuvadis_chr1'
pheno_filename = '../data/geuvadis_test_data/Geuvadis_CEU_YRI_Expr.txt'
anno_filename = '../data/geuvadis_test_data/Geuvadis_CEU_YRI_Annot.txt'

bim,genotype_mat = load_genotypes_plink(geno_prefix)
kinship = generate_kinship(genotype_mat)

bim,fam,bed = limix.io.read_plink(geno_prefix)
fam.set_index('iid',inplace=True)

phenotype_df = pd.read_csv(pheno_filename,sep='\t',index_col=0)
annotation_df = pd.read_csv(anno_filename,sep='\t',index_col=1)
phen_mapping_df = pd.read_csv('../data/geuvadis_test_data/Geuvadis_CEU_gte.txt',sep='\t',header=None,names=['ID','Sample'],index_col=0)

Reading ../data/geuvadis_test_data/Geuvadis_chr1.bim... done (0.00 s)
Reading ../data/geuvadis_test_data/Geuvadis_chr1.fam... done (0.00 s)
Reading ../data/geuvadis_test_data/Geuvadis_chr1.bed... done (0.00 s)


In [90]:
phenotype_df.head()
annotation_df.head()
bim.head()
# print kinship
# print annotation.head()
phen_mapping_df.shape
# print phenotype_df.shape
# phenotype_df.head()

(90, 1)

In [156]:
#indices for relevant snps in genotype matrix
gene_id = 'ENSG00000160075'
gene_id = 'ENSG00000268575'
chrom = str(annotation.loc[gene_id,'Chromosome Name'])
start = annotation.loc[gene_id,'Gene Start (bp)']
end = annotation.loc[gene_id,'Gene End (bp)']
ws = 50000
cis = bim.query("chrom == '%s' & pos > %d & pos < %d" % (chrom, start-ws, end+ws))
snp_idxs = cis['i'].values
snp_names = cis['snp'].values
    
len(snps)

#indices for relevant individuals in genotype matrix
individual_ids = list(set(fam.index)&set(phen_mapping_df.index))
individual_idxs = fam.loc[individual_ids,'i'].values

#subset genotype matrix and kinship matrix
snps = bed[snp_idxs,:].compute().transpose()
snps = snps[individual_idxs,:]

kinship_mat = kinship[sample_idxs,:][:,sample_idxs]

#map individual_ids to samples
sample_ids = phen_mapping_df.loc[individual_ids,'Sample'].values
phenotype = np.array(phenotype_df.loc[gene_id,sample_ids])


In [157]:
print snps.shape
print kinship_mat.shape
print phenotype.shape

(90, 16)
(90, 90)
(90,)


In [164]:
# LMM = limix.qtl.qtl_test_lmm(snps, phenotype,K=kinship_mat)
LMM = limix.qtl.qtl_test_lmm(snps, phenotype)

In [165]:
results = pd.DataFrame(index = range(len(snp_names)),columns=['PValue','SNPName','Beta'])

results['SNPName']=snp_names
results['PValue'] = LMM.getPv()[0]
results['Beta'] = LMM.getBetaSNP()[0]
results.sort_values('PValue')

,PValue,SNPName,Beta
11,0.000019,snp_1_1686040,-0.467989
5,0.000022,snp_1_1616058,0.431096
4,0.000024,snp_1_1615348,0.424067
12,0.000028,snp_1_1693726,0.447076
14,0.000028,snp_1_1703303,0.447076
15,0.000028,snp_1_1704899,0.447076
8,0.000290,snp_1_1658343,-0.356234
6,0.001055,snp_1_1635619,-0.354039
10,0.013081,snp_1_1665702,0.315880
13,0.036100,snp_1_1699201,-0.288084


In [166]:
test_output = pd.read_csv('../data/geuvadis_test_data/TestOutput/Cis-CEU-eQTLsFDR0.05.txt',sep='\t')
test_output[test_output['ProbeName']==gene_id][['PValue','SNPName','Beta (SE)']]

,PValue,SNPName,Beta (SE)
6,3.858690e-07,snp_1_1686040,0.5051299 (0.0920007)
7,1.689560e-06,snp_1_1615348,-0.4795281 (0.0935446)
8,1.691240e-06,snp_1_1703303,-0.4803691 (0.0934956)
9,2.118280e-06,snp_1_1616058,-0.4757112 (0.0937658)
10,2.647870e-06,snp_1_1722932,-0.4723416 (0.0939592)
11,2.799600e-06,snp_1_1704899,-0.4708997 (0.0940414)
12,3.300090e-06,snp_1_1693726,-0.4681373 (0.094198)
13,7.741900e-06,snp_1_1635619,0.4524514 (0.095065)
17,2.702350e-05,snp_1_1837025,-0.426936 (0.0963968)
22,3.433910e-05,snp_1_1658343,0.4219881 (0.096644)
